### 1. Import libraries

In [ ]:
import os
import pandas as pd
import json
import ast


### 2. Marge Data

In [ ]:
path=r"C:\Users\Debasish Das\Desktop\Internship Project\Hybrid_rccomendation_System\data"
all_files=os.listdir(path)


#get_all data
df=pd.read_csv(os.path.join(path,all_files[0]))
df['users'] = df['users'].apply(ast.literal_eval)
user=pd.json_normalize(df["users"])

#inspire data
df=pd.read_csv(os.path.join(path,all_files[1]))
df['posts'] = df['posts'].apply(ast.literal_eval)
inspire=pd.json_normalize(df["posts"])

#liked data

df=pd.read_csv(os.path.join(path,all_files[2]))
df['posts'] = df['posts'].apply(ast.literal_eval)
liked=pd.json_normalize(df["posts"])

#rating data

df=pd.read_csv(os.path.join(path,all_files[3]))
df['posts'] = df['posts'].apply(ast.literal_eval)
rated=pd.json_normalize(df["posts"])


#view data  

df=pd.read_csv(os.path.join(path,all_files[4]))
df['posts'] = df['posts'].apply(ast.literal_eval)
view=pd.json_normalize(df["posts"])



In [ ]:
# df=pd.concat([user,inspire,liked,rated,summary,view], axis=1)

### 3. User data

In [ ]:
user["latitude"]

In [ ]:
user["id"].is_unique

In [ ]:
user.columns

In [ ]:
user=user.drop(columns=[ 'last_name', 'username', 'email',
       'profile_url','website_url', 'instagram-url', 'youtube_url',
       'tictok_url', 'isVerified', 'referral_code', 'has_wallet', 'last_login', 'is_online', 'latitude', 'longitude'])

In [ ]:
user.columns

### 4. inspire

In [ ]:
inspire.columns

In [ ]:
inspire=inspire.drop(columns=['slug', 'identifier', 'contract_address', 'chain_id',
       'chart_url', 'is_locked', 'created_at', 'last_name',
       'bookmarked', 'thumbnail_url',
       'gif_thumbnail_url', 'picture_url', 'category.image_url', 'baseToken.address',
       'baseToken.name', 'baseToken.symbol', 'baseToken.image_url'])

In [ ]:
inspire.columns

### 5. liked data

In [ ]:
liked.head()

In [ ]:
liked.columns

In [ ]:
liked=liked.drop(columns=['slug','identifier', 'contract_address', 'chain_id',
       'chart_url', 'is_locked', 'created_at', 'last_name', 'bookmarked', 'thumbnail_url',
       'gif_thumbnail_url', 'following', 'picture_url', 'category.image_url', 'baseToken.address',
       'baseToken.name', 'baseToken.symbol', 'baseToken.image_url'])

### 6. Rating

In [ ]:
rated.head()

In [ ]:
rated.columns

In [ ]:
rated=rated.drop(columns=['slug', 'identifier', 'contract_address', 'chain_id',
       'chart_url', 'is_locked', 'created_at', 'first_name', 'last_name', 'bookmarked', 'thumbnail_url',
       'gif_thumbnail_url', 'following', 'picture_url', 
        'category.image_url', 'baseToken.address',
       'baseToken.name', 'baseToken.symbol', 'baseToken.image_url'])

### 8. view

In [ ]:
view.head()

In [ ]:
view.columns

In [ ]:
view=view.drop(columns=['slug', 'identifier', 'contract_address', 'chain_id',
       'chart_url', 'is_locked', 'created_at', 'last_name', 'bookmarked', 'thumbnail_url',
       'gif_thumbnail_url', 'following', 'picture_url',
        'category.image_url', 'baseToken.address',
       'baseToken.name', 'baseToken.symbol', 'baseToken.image_url'])

In [ ]:
df=pd.merge(liked,view,on=["id","title",'comment_count', 'upvote_count',
       'view_count', 'exit_count', 'rating_count', 'average_rating',
       'share_count','video_link','upvoted','category.name', 'category.count',
       'category.description',"username",'category.id'],how="outer")
df=pd.merge(df,rated,on=["id","title",'comment_count', 'upvote_count',
       'view_count', 'exit_count', 'rating_count', 'average_rating',
       'share_count','upvoted','category.name', 'category.count',
       'category.description',"username","video_link",'category.id'],how="outer")
df = pd.merge(df, inspire, on=["id","title",'comment_count', 'upvote_count',
       'view_count', 'exit_count', 'rating_count', 'average_rating',
       'share_count','upvoted','category.name', 'category.count','video_link',
       'category.description',"username",'category.id'], how="outer",suffixes=('_left', '_right'))

In [ ]:
df["username"]

In [ ]:
df.drop(columns=["first_name_x","first_name_y","post_summary_y","post_summary_left","post_summary_right","first_name"],inplace=True)

In [ ]:
df=pd.merge(df,user, on=["id"],how="outer")

In [ ]:
df["username"]

In [ ]:
df.drop(columns=["share_count_x"],inplace=True)

In [ ]:
# df=df.fillna(0)

In [ ]:
merged_data=df

In [ ]:
# # Fill missing values for text-based columns using .loc
# merged_data.loc[:, 'post_summary_x'] = merged_data['post_summary_x'].fillna('')
# merged_data.loc[:, 'category.description'] = merged_data['category.description'].fillna('')

In [ ]:
merged_data.columns

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert relevant columns to numeric, forcing errors to NaN
merged_data['comment_count'] = pd.to_numeric(merged_data['comment_count'], errors='coerce').fillna(0)
merged_data['upvote_count'] = pd.to_numeric(merged_data['upvote_count'], errors='coerce').fillna(0)
merged_data['view_count'] = pd.to_numeric(merged_data['view_count'], errors='coerce').fillna(0)
merged_data['share_count_y'] = pd.to_numeric(merged_data['share_count_y'], errors='coerce').fillna(0)
merged_data["follower_count"]= merged_data["follower_count"].fillna('')
merged_data["following_count"]=merged_data["following_count"].fillna('')

# Calculate engagement score
merged_data['engagement_score'] = (
    merged_data['comment_count'] +
    merged_data['upvote_count'] +
    merged_data['view_count'] +
    merged_data['share_count_y']
)

# Ensure all text columns are strings before concatenation
merged_data['title'] = merged_data['title'].astype(str)
merged_data['post_summary_x'] = merged_data['post_summary_x'].astype(str)
merged_data['category.name'] = merged_data['category.name'].astype(str)
merged_data['category.description'] = merged_data['category.description'].astype(str)
merged_data["role"]=merged_data["role"].astype(str)
merged_data["follower_count"]=merged_data["follower_count"].astype(str)
merged_data["bio"]=merged_data["bio"].astype(str)
merged_data["following_count"]=merged_data["following_count"].astype(str)
merged_data["is_verified"]=merged_data["is_verified"].astype(str)
merged_data["category.name"]=merged_data["category.name"].astype(str)


# Combine text-based columns to create a metadata summary for the content
merged_data['metadata'] = (
    merged_data['title'] + ' ' +
    merged_data['post_summary_x'] + ' ' +
    merged_data['category.name'] + ' ' +
    merged_data['category.description']
)


# ## colabrotive columns to create a metadata
# merged_data['colabrotive'] = (
#     merged_data["role"] + ' ' +
#     merged_data["follower_count"]+ ' ' +
#     merged_data["bio"] + ' ' +
#     merged_data["following_count"]+ ' ' +
#     merged_data["is_verified"]+' '+
#     merged_data["category.name"]
# )




In [ ]:
df["category.name"]=df["category.name"].fillna('')

### 9. Content Based 

In [ ]:
# TF-IDF Vectorization
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(merged_data["metadata"])

# Compute Cosine Similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Function to recommend videos
def recommend_content_based(video_id, cosine_sim, posts_df):
    idx = posts_df[posts_df["id"] == video_id].index[0]
    similarity_scores = list(enumerate(cosine_sim[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similar_videos = [posts_df.iloc[i[0]]["video_link"] for i in similarity_scores[0:6]]
    return similar_videos

# Example Recommendation
video_id = 1242
recommended_videos = recommend_content_based(video_id, cosine_sim, merged_data)
print("Recommended Videos:", recommended_videos)


### 10. Collabrotive 

In [ ]:
from sklearn.decomposition import TruncatedSVD
import numpy as np

# Create a User-Item Interaction Matrix
user_item_matrix = merged_data.pivot_table(index="username", columns="id", values="engagement_score", fill_value=0)

# Apply SVD
svd = TruncatedSVD(n_components=10)
user_factors = svd.fit_transform(user_item_matrix)
item_factors = svd.components_

# Function to recommend based on SVD
def recommend_collaborative(user, user_item_matrix, user_factors, item_factors,df):
    user_idx = user_item_matrix.index.get_loc(user)
    user_vector = user_factors[user_idx]
    scores = np.dot(user_vector, item_factors)
    top_item_indices = np.argsort(scores)[::-1][:5]
    ids=user_item_matrix.columns[top_item_indices]
    k=merged_data[merged_data["id"].isin(ids)]
    return k[["username","id","title","video_link","engagement_score"]].to_dict(orient="records")

# Example Recommendation
user = "Rishwanth"
recommended_videos = recommend_collaborative(user, user_item_matrix, user_factors, item_factors,merged_data)
print("Recommended Videos:", recommended_videos)

### 11 Hybrid Recomendation 

In [ ]:
def recommend_new_user_videos(merged_data):
    # 1. Top categories by views
    top_categories = (
        merged_data.groupby("category.name")["view_count"]
        .sum()
        .sort_values(ascending=False)
        .head(6)
        .index
    )

    # Get top videos from popular categories
    category_recommendations = (
        merged_data[merged_data["category.name"].isin(top_categories)]
        .sort_values(by="view_count", ascending=False)
        .head(6)
    )

    # 2. Top-rated videos
    top_rated_videos = (
        merged_data.sort_values(by="average_rating", ascending=False)
        .head(6)
    )

    # 3. Most commented videos
    most_commented_videos = (
        merged_data.sort_values(by="comment_count", ascending=False)
        .head(6)
    )

    # 4. Top viewed videos overall
    top_viewed_videos = (
        merged_data.sort_values(by="view_count", ascending=False)
        .head(6)
    )

    # Combine all recommendations
    recommendations = pd.concat(
        [category_recommendations, top_rated_videos, most_commented_videos, top_viewed_videos]
    ).drop_duplicates(subset="id")

    # Convert to a list of dictionaries
    return recommendations[["id", "title", "category.name", "video_link", "view_count", "average_rating", "comment_count"]].sample(5).to_dict()



In [ ]:
def recommend_hybrid(user, user_item_matrix, user_factors, item_factors, cosine_sim, posts_df):
    if user in user_item_matrix.index:  
        conent= recommend_collaborative(user, user_item_matrix, user_factors, item_factors)
        user_based=recommend_collaborative(user, user_item_matrix, user_factors)
        recomendation=pd.concat([conent,user_based])
    else:  
        return recommend_new_user_videos(merged_data)


user = "new_user"  
recommended_videos = recommend_hybrid(user, user_item_matrix, user_factors, item_factors, cosine_sim, merged_data)
print("Recommended Videos:", recommended_videos)